In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors
import glob

from numba import njit, vectorize, float32

from typing import Callable, Optional, Union

import hoomd
import gsd.hoomd

import sys
import time
import pickle
import gc
import pathlib
import os

import signac

from scipy import optimize

In [2]:
from monk import nb, prep, pair, render, utils, grid, workflow
import freud

parent = pathlib.Path(os.getcwd()).parent / "config.yaml"
config = workflow.get_config(parent.as_posix())

# mpl.rcParams["text.usetex"] = True
# mpl.rcParams["text.latex.preamble"].append(r'\usepackage{xfrac}')

In [3]:
project: signac.Project = signac.get_project(root=config['root'])
project.doc

{'dt': 0.001, 'usteps': 1000, 'avail_seed': 17, 'temp_steps': 20, 'step_unit': 1000, 'equil_time': 40, 'run_time': 10, 'max_alpha_time': 1000, 'alpha_iters': 20, '_status': {}}

In [5]:
for job in project:
    print(job, job.doc["temp_steps"])

b3fd994a7eac1b88cb4fe56e9e5435bf [0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.30000000000000004, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.65, 0.7000000000000001, 0.75, 0.8, 0.8500000000000001, 0.9, 0.9500000000000001, 1.0]
622125796b71d60351b287db538be2eb [0.30000000000000004, 0.6000000000000001, 0.9000000000000001, 1.2000000000000002, 1.5, 1.8000000000000003, 2.1, 2.4000000000000004, 2.7, 3.0, 3.3000000000000003, 3.6000000000000005, 3.9000000000000004, 4.2, 4.5, 4.800000000000001, 5.1000000000000005, 5.4, 5.7, 6.0]
3528d9233bb0188fa5c5af4cd0d46ebb [0.275, 0.55, 0.8250000000000002, 1.1, 1.375, 1.6500000000000004, 1.9250000000000003, 2.2, 2.475, 2.75, 3.0250000000000004, 3.3000000000000007, 3.575, 3.8500000000000005, 4.125, 4.4, 4.675000000000001, 4.95, 5.2250000000000005, 5.5]
a03bdc2a5be70a1e699a3f825fb8877b [0.07500000000000001, 0.15000000000000002, 0.22500000000000003, 0.30000000000000004, 0.375, 0.45000000000000007, 0.525, 0.6000000000000001, 0.675, 0.75, 0